# Database class

In [19]:
import pymysql
from PIL import Image
import numpy as np
import cv2
from pathlib import Path
import re

In [122]:
""" ======================================================
class database : to handel connection and quarries 
methods :
 * create_table : to create tables with at least one column in it. 
 * add_column : to add new columns or column to a specific table you created before. 
 * drop_table : to delete table from a database. 
 * drop_column : to delete column. 
 * ulter_table : to modify a table. 
 *insert : to insert new data
 * delete_all : to delete all data
 * delete : to delete based on condition 
 * read_all_data : to select all data
 *read : to read based on condition 
"""

' ======================================================\nclass database : to handel connection and quarries \nmethods :\n * create_table : to create tables with at least one column in it. \n * add_column : to add new columns or column to a specific table you created before. \n * drop_table : to delete table from a database. \n * drop_column : to delete column. \n * ulter_table : to modify a table. \n *insert : to insert new data\n * delete_all : to delete all data\n * delete : to delete based on condition \n * read_all_data : to select all data\n *read : to read based on condition \n'

In [119]:
class database :
    def __init__(self , database_name) :
        #create connection to the database
        self.databaseName = database_name
        self.db = pymysql.connect('127.0.0.1','root','',database_name)
        self.cursor = self.db.cursor()
     
    
    def create_table(self , table_name , column_name , column_type , constraint='') :
        """
        this fuction is used to create table :
         *table_name: name of the table.
         *column_name: basic column or any column you want.
         *column_type: the type of the column.
         *constraint: like not null  string, defult:no constraint.
          output : table with just one column ".
        """
        try :
            sql = "create table " + table_name +"( " +column_name + " " + column_type + " " + constraint + " )"  #sql querry
            self.cursor.execute(sql) 
            print(table_name +" is created sucessfully")
        except :
            print("error")
            
            
    def add_columns(self,table_name ,column_names,column_types) :
        """
        to add new_columns 
         *table_name: the name of the table you want to modify
         *column_names: list of the new column names you want to add
         *column_types: list of types of the columns you added in the same order
         *output :table with this new columns 
        """        
        try :
            for i in range(0,len(column_names)) :
                sql = 'ALTER TABLE '+ table_name+ ' ADD ' + column_names[i]+ " " + column_types[i]
                self.cursor.execute(sql)
            print("table is updated successfully")    
        except :
            print("error")

            
    def drop_table(self , table_name) :
        """ is used to drop / delete table
        *table_name : string 
        """
        try :
            sql = 'DROP TABLE ' + table_name;
            self.cursor.execute(sql)
            print(table_name , ' is dropped successfully ')
        except :
            print('error')
         
        
    def drop_columns(self , table_name , column_names) :
        """
        is used to delete column/s :
        *table_name: the table name (string)
        *column_names : enter the column/s name (list
        """
        try :
            for i in column_names :
                sql = 'ALTER TABLE '+ table_name + ' DROP COLUMN ' + column_name
                print(sql)
                self.cursor.execute(sql)
            print(column_name , 'dropped successfully ')
        except :
            print('error')     
            
            
            
    def ulter_column(self , table_name , old_column_name , new_column_name='' , constarint = '' , data_type = None ) :
        """it used to modify column data_type , add constraint "
        "it takes 5 args :
        *table_name 
        *old_column_name 
        *new_column_name (if u eant to change it )"
        *constraint (if u want also )"
        *data_type """
        
        try :
            if data_type != None :
                sql = 'alter table ' + table_name + ' modify column ' + old_column_name +" "+  new_column_name + " " +  data_type + " " + constarint
                self.cursor.execute(sql)
            print("column is ultered successfully")    
        except :
            print("error")
            
    
    def insert(self , table_name, l ) :
        """
        to add new row to the table 
        *table_name : the name of the table 
        *l : list of the given values and it should be 6 in length 
        
        """
        sql = 'insert into ' + table_name + ' values(%s , %s , %s , %s , %s , %s) '
        try :
            
            self.cursor.execute(sql, tuple(l))
            self.db.commit()
            print("row added ")
        except :
            print("error")
            
            
    def read_all_data(self , table_name) :
        "to read all data "
        
        sql = 'select* from ' + table_name
        try :
            self.cursor.execute(sql)
            row = self.cursor.fetchall()
            return row
        except :
            print('error')
            
    def read(self , table_name , column , condition) :
        """
        read the row that match the key and condition
        *table_name : the name of the table
        *key : which column u want 
        *condition : = where the key equal to the condition
        output : return row/s that match/es the condition 
        """
        sql = 'SELECT * FROM '+ table_name + ' WHERE ('+ column +  ' = %s )'
        try :
            self.cursor.execute(sql ,condition)
            row = self.cursor.fetchall()
            return row
        except :
            print("error")
            
            
    def delete_row(self , table_name , key , condition) :
        sql = 'DELETE FROM ' + table_name + ' WHERE ' + key +' = %s' 
        try :
            self.cursor.execute(sql , (condition ,))
            self.db.commit()
            print("row deleted ")
        except :
            print("error")
    
    def delete_all(self , table_name) :
        "clear all the table"
        sql = 'DELETE * FROM ' + table_name
        try :
            self.cursor.execute(sql)
            print("data is cleared ")
            
        except :
            print("errro")
    def close(self):
        """
        close the database connection
        """
        self.db.close()
   

In [90]:
def fill_data(*args):
    result = list()
    for item in args :
        result.append(item)
    return result


def convert_to_string(l) :
    return ' '.join(l) 


def main():
    main_dir = 'D:\\Graduation Project\\AD - Test Data\\MRI'
    p = Path(main_dir)
    #2 - get the list of the data (ids of patients)
    list_of_data = [x.name for x in p.iterdir() if x.is_dir()]

    data = list() # to append the data I need to insert one row 
    for _id in list_of_data : #uterate through ids
        sub_dir1 = main_dir + "\\" + l 
        path_id = Path(sub_dir1) 
        for ray_type in path_id.iterdir() : #iterate through ray_type
            if ray_type.is_dir() :
                for date in ray_type.iterdir() : #iterate through date 
                    for num in date.iterdir() : 
                        x = [x.name for x in num.iterdir() if x.glob('**/*.dcm')] #get the name of the 
                        #get a list of the data I need to insert into the table 
                        data = fill_data(main_dir ,_id, ray_type.name, date.name , num.name , convert_to_string(x))
                        obj.insert('patients' , data)
                        #clear the list for a new row 
                        data.clear()
                            

In [91]:
obj = database('data')
main()

row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 
row added 

In [93]:
x = d.read('patients' , 'id' , '011_S_0003')

SELECT * FROM patients WHERE (id = %s )
hhhh


In [94]:
x[0]

('D:\\Graduation Project\\AD - Test Data\\MRI',
 '011_S_0003',
 'Axial_PD-T2_TSE',
 '2005-09-01_10_46_12.0',
 'S9126',
 'ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115036667_1_S9126_I7053.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115036980_10_S9126_I7053.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115037480_100_S9126_I7062.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115037777_101_S9126_I7062.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115038074_102_S9126_I7062.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115038386_103_S9126_I7062.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115038683_104_S9126_I7062.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115038996_105_S9126_I7062.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115039308_106_S9126_I7062.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115039605_107_S9126_I7062.dcm ADNI_011_S_0003_MR_Axial_PD-T2_TSE__br_raw_20050901115039917_108_S9126_I

In [96]:
d.insert('patients' , ["sss" , "sss" , "ssss" , "sssss" , "sssss" , "sssss"])

row added 


In [120]:
obj = database('data')

In [121]:
obj.delete_row('patients' , 'id' , 'sss')

DELETE FROM patients WHERE id = %s
start
llll
row deleted 
